In [69]:
import pandas as pd
import hashlib
import numpy
import os

pd.set_option('display.max_rows', 200)

# Task 1

In [2]:
df = pd.read_csv('100k.txt', sep='\t', header=None, names=['Subject', 'Property','Object'])
df['Object'] = df.Object.str.rstrip(" .")
# df

In [3]:
df["Property"] = df.Property.str.split(":").apply(lambda x: x[1])
df["Object"] = df.Object.str.split(":").apply(lambda x: x[1] if len(x)>1 else x[0])
df["Subject"] = df.Subject.str.split(":").apply(lambda x: x[1])

In [4]:
property_dicts = {}
for prop, df_part in df.groupby('Property'):
    print(prop)
    print(df_part.head(2))
    print("========================")
    property_dicts[prop] = df_part[["Subject","Object"]]

Location
    Subject  Property   Object
281   User3  Location  City215
404   User4  Location    City2
actor
         Subject Property Object
95279  Product19    actor  User3
95280  Product19    actor  User5
age
    Subject Property     Object
46    User1      age  AgeGroup5
673   User7      age  AgeGroup5
aggregateRating
         Subject         Property Object
96924  Retailer0  aggregateRating      4
97230  Retailer3  aggregateRating      4
artist
         Subject Property   Object
94720   Product3   artist  User114
96024  Product59   artist  User731
author
        Subject Property   Object
94760  Product6   author   User90
94761  Product6   author  User175
award
         Subject Property                                             Object
96523  Product77    award                                           Dniester
96645  Product83    award  power killdeer deodorant voiceless codeine too...
birthDate
     Subject   Property      Object
1007   User9  birthDate  1974-02-09
1493  User12  

In [5]:
numeric_objects = []
for k_prop in property_dicts.keys():
    if property_dicts[k_prop]['Object'].str.isnumeric().all():
        numeric_objects.append(k_prop)
        property_dicts[k_prop]['Object'] = property_dicts[k_prop]['Object'].apply(lambda x: int(x))

/var/folders/t4/sp85qyw93319q2vj6jvsjg2r0000gp/T/ipykernel_35111/2289272882.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_dicts[k_prop]['Object'] = property_dicts[k_prop]['Object'].apply(lambda x: int(x))


In [6]:
numeric_objects

['aggregateRating',
 'bookEdition',
 'contentRating',
 'contentSize',
 'duration',
 'eligibleQuantity',
 'faxNumber',
 'hits',
 'isbn',
 'movement',
 'numberOfPages',
 'openingHours',
 'opus',
 'price',
 'printColumn',
 'printEdition',
 'printPage',
 'printSection',
 'rating',
 'record_number',
 'serialNumber',
 'telephone',
 'totalVotes',
 'userId',
 'wordCount']

In [7]:
# so for each user he has an id

df[df.Subject.apply(lambda x: x[:4] == "User")].Subject.nunique()

1000

In [8]:
property_dicts['userId'].Subject.nunique()

1000

In [9]:
user_ID_mapping = dict(property_dicts['userId'].values)
user_ID_mapping['User0']

1806723

In [10]:
# so we can fill the user with the user id
user_properties = []
user_dual_properties = []

for k_prop in property_dicts.keys():
    if property_dicts[k_prop]['Subject'].apply(lambda x: x[:4] == "User").all():
        if k_prop == "userId":
            print(k_prop)
            continue
        user_properties.append(k_prop)
        property_dicts[k_prop]['Subject'] = property_dicts[k_prop]['Subject']\
                    .apply(lambda x: user_ID_mapping[x])
        if property_dicts[k_prop]['Object'].apply(lambda x: x[:4] == "User").all():
            user_dual_properties.append(k_prop)
            property_dicts[k_prop]['Object'] = property_dicts[k_prop]['Object']\
                    .apply(lambda x: user_ID_mapping[x])

userId


In [11]:
property_dicts['follows']

,Subject,Object
12,1806723,2680384
13,1806723,2211675
14,1806723,5335024
15,1806723,2915528
16,1806723,2004500
...,...,...
88905,3143980,1986609
88906,3143980,5131970
88907,3143980,1358408
88908,3143980,1661104


In [12]:
user_properties

['Location',
 'age',
 'birthDate',
 'familyName',
 'follows',
 'friendOf',
 'gender',
 'givenName',
 'jobTitle',
 'likes',
 'makesPurchase',
 'nationality',
 'subscribes']

In [13]:
user_dual_properties

['follows', 'friendOf']

In [14]:
for _, row in property_dicts['follows'].iterrows():
    print(row.index)
    break

Index(['Subject', 'Object'], dtype='object')


# Task 2

**Hash join**

In [15]:
def partition(table, join_key, hash_function):
    table['hash_key'] = table[join_key].apply(hash_function)
    hash_table = dict(tuple(table.groupby('hash_key')))
    table.drop('hash_key', axis=1, inplace=True)
    return hash_table


def probe(hash_table, join_key1, table2, join_key2, hash_function):
    dfs_to_join = []
    for _, row in table2.iterrows():
        hash_key = hash_function(row[join_key2])
        if hash_key in hash_table:
            df_hash = hash_table[hash_key].copy()
            df_hash = df_hash[df_hash[join_key1] == row[join_key2]]
            for col in row.index:
                df_hash[col + '_2'] = row[col]
            dfs_to_join.append(df_hash)
    joined_tables = pd.concat(dfs_to_join,axis=0)
    return joined_tables

def hash_join(table1, table2, join_key1, join_key2, hash_function=hash, join_type='inner'):

    if join_type == 'right':
        temp = table1
        table1 = table2
        table2 = temp
        temp_key = join_key1
        join_key1 = join_key2
        join_key2 = temp_key
    
    hash_table = partition(table1, join_key1, hash_function)
    joint = probe(hash_table, join_key1, table2, join_key2, hash_function).drop('hash_key', axis=1)
    
    if join_type in ['right', 'left']:
        return pd.concat([joint, table1[~table1[join_key1].apply(lambda x: x in joint[join_key1].values)]],axis=0)
    
    return joint

In [16]:
hash_join(property_dicts['Location'], property_dicts['parentCountry'],'Object','Subject',join_type='inner').sort_values('Object')

,Subject,Object,Subject_2,Object_2
1771,8163087,City0,City0,Country20
54114,4217937,City0,City0,Country20
55056,2040846,City0,City0,Country20
57123,2225099,City0,City0,Country20
57851,2796600,City0,City0,Country20
...,...,...,...,...
46840,7896679,City94,City94,Country3
60268,2140616,City94,City94,Country3
33210,4055499,City94,City94,Country3
41594,2479861,City96,City96,Country1


**Sort-merge join**

In [17]:
def sort_merge_join(table1, table2, join_key1, join_key2,join_type='inner'):

    if join_type == 'right':
        temp = table1
        table1 = table2
        table2 = temp
        temp_key = join_key1
        join_key1 = join_key2
        join_key2 = temp_key

    sorted_table1 = table1.sort_values(join_key1)
    sorted_table2 = table2.sort_values(join_key2)
    
    pointer1 = 0 
    pointer2 = 0
    result = []
    
    while True:
        condition1 = pointer1 < len(sorted_table1) 
        condition2 = pointer2 < len(sorted_table2)
        
        if join_type=='inner' and not (condition1 and condition2):
            break
        
        if join_type in ['right','left']:
            if condition1 and not condition2:
                table1_remaining = sorted_table1[~sorted_table1[join_key1].isin(table2.sort_values(join_key2))]
                indices, rowSeries = zip(*table1_remaining.add_suffix('_1').iterrows())
                result.extend(list(rowSeries))
                break
            elif not condition1:
                break
        
        value1 = sorted_table1.iloc[pointer1][join_key1]
        value2 = sorted_table2.iloc[pointer2][join_key2]

        if value1 == value2:
            result.append(pd.concat([sorted_table1.iloc[pointer1].add_suffix('_1'), sorted_table2.iloc[pointer2].add_suffix('_2')]))
            
            skip_condition1 = (pointer2 < len(sorted_table2)-1) and (sorted_table2.iloc[pointer2+1][join_key2] != value2)
            skip_condition2 = (pointer1 < len(sorted_table1)-1) and (sorted_table1.iloc[pointer1+1][join_key1] != value1)
            
            if skip_condition1:
                pointer1 += 1
            if skip_condition2:
                pointer2 += 1
                
        elif value1 < value2:
                pointer1 += 1
        else:
                pointer2 += 1
#     print(result[0])
#     print(result[-1])
    return pd.DataFrame(result)


In [18]:
merged = sort_merge_join(property_dicts['Location'], property_dicts['parentCountry'],'Object','Subject',join_type='inner')

In [19]:
merged

,Subject_1,Object_1,Subject_2,Object_2
0,1548262,City0,City0,Country20
1,9278057,City0,City0,Country20
2,5797591,City0,City0,Country20
3,4303655,City0,City0,Country20
4,9291031,City0,City0,Country20
...,...,...,...,...
382,7896679,City94,City94,Country3
383,2140616,City94,City94,Country3
384,4055499,City94,City94,Country3
385,2479861,City96,City96,Country1


In [20]:
indices, rowSeries =zip(*(property_dicts['Location'].sort_values('Object').iloc[96:98]).iterrows())
rowSeries

(Subject    4810084
 Object     City106
 Name: 44024, dtype: object,
 Subject    1890641
 Object     City109
 Name: 3155, dtype: object)

In [21]:
property_dicts['parentCountry'].sort_values('Subject').iloc[10:13]

,Subject,Object
9073,City107,Country2
9074,City108,Country1
9075,City109,Country17


In [22]:
hashed = hash_join(property_dicts['Location'], property_dicts['parentCountry'],'Object','Subject',join_type='inner')
merged = sort_merge_join(property_dicts['Location'], property_dicts['parentCountry'],'Object','Subject',join_type='inner')
(~hashed['Subject'].isin(merged[0])).sum()

KeyError: 0

In [ ]:
missing = hashed[(~hashed['Subject'].isin(merged[0]))]
missing

In [ ]:
pd.set_option('display.max_rows', None)

# Task 1 with 'watdiv.10M.nt' Big Data File

In [75]:
# we read the data in batch of 100. so the memory is never overflooded regardless of the data size.
# also, after reading the data, we make one file for each property and neglect all the other properties
# that are not necessary to save memory size and processing time.

file_path = 'watdiv.10M.nt'
batch_size = 100
file_path_follows = 'follows.txt'
file_path_friendOf = 'friendOf.txt'
file_path_likes = 'likes.txt'
file_path_hasReview = 'hasReview.txt'

# Function to delete a file if it exists
def delete_if_exists(file_path):
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Deleted '{file_path}'.")
    else:
        print(f"'{file_path}' does not exist.")

# Delete the files if they exist
delete_if_exists(file_path_follows)
delete_if_exists(file_path_friendOf)
delete_if_exists(file_path_likes)
delete_if_exists(file_path_hasReview)


# Read the file in chunks using pd.read_csv() and process each chunk
# No need to extract the property in seperated files
# Because we have each seperate file for each property type
for df_chunk in pd.read_csv(file_path, chunksize=batch_size, sep='\t', header=None, names=['Subject', 'Property', 'Object']):
    df_chunk['Property'] = df_chunk['Property'].str.extract(r'[#/]([^#/>]+)>\s*$')
    filtered_df = df_chunk.loc[df_chunk['Property'].str.contains('follows|friendOf|likes|hasReview')]
    if len(filtered_df):
        if (filtered_df['Property'] == 'follows').sum():
            filtered_df[filtered_df['Property'] == 'follows'][['Subject', 'Object']].to_csv(file_path_follows, mode='a', index=False, header=False)
        
        if (filtered_df['Property'] == 'friendOf').sum():
            filtered_df[filtered_df['Property'] == 'friendOf'][['Subject', 'Object']].to_csv(file_path_friendOf, mode='a', index=False, header=False)
        
        if (filtered_df['Property'] == 'likes').sum():
            filtered_df[filtered_df['Property'] == 'likes'][['Subject', 'Object']].to_csv(file_path_likes, mode='a', index=False, header=False)
        
        if (filtered_df['Property'] == 'hasReview').sum():
            filtered_df[filtered_df['Property'] == 'hasReview'][['Subject', 'Object']].to_csv(file_path_hasReview, mode='a', index=False, header=False)


Deleted 'follows.txt'.
Deleted 'friendOf.txt'.
Deleted 'likes.txt'.
Deleted 'hasReview.txt'.


In [78]:
list_of_dfs = ['follows', 'friendOf', 'likes', 'hasReview']
property_dicts = {}

for prop in list_of_dfs:
    property_dicts[prop] = pd.read_csv(f'{prop}.txt', header=None, names=['Subject','Object'])
    property_dicts[prop]['Object'] = property_dicts[prop].Object.str.rstrip(" .")
    property_dicts[prop]['Object'] = property_dicts[prop]['Object'].str.extract(r'[#/]([^#/>]+)>\s*$')
    property_dicts[prop]['Subject'] = property_dicts[prop]['Subject'].str.extract(r'[#/]([^#/>]+)>\s*$')

property_dicts['follows']

,Subject,Object
0,User10000,User10146
1,User10000,User12191
2,User10000,User13638
3,User10000,User15527
4,User10000,User15778
...,...,...
3289302,User9,User26462
3289303,User9,User30532
3289304,User9,User51126
3289305,User9,User90573


In [ ]:
property_dicts = {}
for prop, df_part in df.groupby('Property'):
    print(prop)
    print(df_part.head(2))
    print("========================")
    property_dicts[prop] = df_part[["Subject","Object"]]